# Problem: Implement a CNN for CIFAR-10 in PyTorch

### Problem Statement
You are tasked with implementing a **Convolutional Neural Network (CNN)** for image classification on the **CIFAR-10** dataset using PyTorch. The model should contain convolutional layers for feature extraction, pooling layers for downsampling, and fully connected layers for classification. Your goal is to complete the CNN model by defining the necessary layers and implementing the forward pass.

### Requirements
1. **Define the CNN Model**:
   - Add **convolutional layers** for feature extraction.
   - Add **pooling layers** to reduce the spatial dimensions.
   - Add **fully connected layers** to output class predictions.
   - The model should be capable of processing input images of size `(32x32x3)` as in the CIFAR-10 dataset.

### Constraints
- The CNN should be designed with multiple convolutional and pooling layers followed by fully connected layers.
- Ensure the model is compatible with the CIFAR-10 dataset, which contains 10 classes.


<details>
  <summary>💡 Hint</summary>
  Add the convolutional (conv1, conv2), pooling (pool), and fully connected layers (fc1, fc2) in CNNModel.__init__.
  <br>
  Implement the forward pass to process inputs through these layers.
</details>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [7]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

AttributeError: 'DataLoader' object has no attribute 'to'

In [3]:
print(train_loader.dataset.classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [9]:
# Define the CNN Model
# TODO: Add convolutional, pooling, and fully connected layers
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5).to('cuda')
        # (32 -> 28)
        self.pool = nn.MaxPool2d(2,2).to('cuda')
        # (28 -> 14)
        self.conv2 = nn.Conv2d(32, 64, 5).to('cuda')
        # (14 -> 10) -> 5
        self.lin1 = nn.Linear(64 * 5 * 5, 120).to('cuda')
        self.lin2 = nn.Linear(120, 64).to('cuda')
        self.lin3 = nn.Linear(64, 10).to('cuda')

    def forward(self, x):
        x = x.to('cuda')
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.reshape(-1, 64 * 5 * 5)
        x = nn.functional.relu(self.lin1(x))
        x = nn.functional.relu(self.lin2(x))
        x = self.lin3(x)
        return x

In [12]:
# Initialize the model, loss function, and optimizer
model = CNNModel().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        images = images.to('cuda')
        labels = labels.to('cuda')
        outputs = model(images).to('cuda')
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 1.2186
Epoch [2/10], Loss: 0.6189
Epoch [3/10], Loss: 0.7245
Epoch [4/10], Loss: 0.7677
Epoch [5/10], Loss: 0.9326
Epoch [6/10], Loss: 0.4851
Epoch [7/10], Loss: 0.8468
Epoch [8/10], Loss: 0.6261
Epoch [9/10], Loss: 0.3266
Epoch [10/10], Loss: 0.9953


In [15]:
# Evaluate on the test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.to('cuda')
        outputs = model(images).to('cuda')
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 68.26%
